In [2]:
import pandas as pd


In [3]:
# gets data from both of the datasets
tract_incarc_rates = pd.read_csv("tract_jail_rP_gP_pall.csv")
acsdata = pd.read_csv("acs2017_census_tract_data.csv")

In [4]:
# merges the data
merged_df = acsdata.merge(tract_incarc_rates, left_on='TractId', right_on='tract', how='inner')


In [5]:
merged_df.head()

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,tract,Name,Incarceration_Rate_rP_gP_pall
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.2,0.0,...,24.5,881,74.2,21.2,4.5,0.0,4.6,1001020100,"Prattville, AL",0.0198
1,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,...,22.2,852,75.9,15.0,9.0,0.0,3.4,1001020200,"Prattville, AL",0.0292
2,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,...,23.1,1482,73.3,21.1,4.8,0.7,4.7,1001020300,"Prattville, AL",0.0086
3,1001020400,Alabama,Autauga County,4267,2001,2266,9.6,80.3,7.1,0.5,...,25.9,1849,75.8,19.7,4.5,0.0,6.1,1001020400,"Prattville, AL",0.0088
4,1001020500,Alabama,Autauga County,9965,5054,4911,0.9,77.5,16.4,0.0,...,21.0,4787,71.4,24.1,4.5,0.0,2.3,1001020500,"Prattville, AL",0.0042


In [6]:
merged_df.columns

Index(['TractId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment', 'tract', 'Name',
       'Incarceration_Rate_rP_gP_pall'],
      dtype='object')

In [7]:
# gets only the important columns
merged_df = merged_df.drop(columns=merged_df.columns.difference(['TractId','State', "TotalPop", "White", "Black", "Native", "Asian", "Hispanic" ,'Incarceration_Rate_rP_gP_pall']))



In [8]:
merged_df.head()

,TractId,State,TotalPop,Hispanic,White,Black,Native,Asian,Incarceration_Rate_rP_gP_pall
0,1001020100,Alabama,1845,2.4,86.3,5.2,0.0,1.2,0.0198
1,1001020200,Alabama,2172,1.1,41.6,54.5,0.0,1.0,0.0292
2,1001020300,Alabama,3385,8.0,61.4,26.5,0.6,0.7,0.0086
3,1001020400,Alabama,4267,9.6,80.3,7.1,0.5,0.2,0.0088
4,1001020500,Alabama,9965,0.9,77.5,16.4,0.0,3.1,0.0042


In [9]:
# drops all rows which do not have an incarceration rate
merged_df = merged_df.dropna(subset=['Incarceration_Rate_rP_gP_pall'])


In [10]:
# estimates the incarceration rates by multiplying demographic percentage by total population of tract and then multiplying by incarceration rate
# I also multiplied by 0.01 to get into decimal percetange from
merged_df.loc[:, 'Asian_Incarcerated'] = merged_df['Asian'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['TotalPop'] * 0.01
merged_df.loc[:, 'White_Incarcerated'] = merged_df['White'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['TotalPop'] * 0.01
merged_df.loc[:, 'Native_Incarcerated'] = merged_df['Native'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['TotalPop'] * 0.01
merged_df.loc[:, 'Black_Incarcerated'] = merged_df['Black'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['TotalPop'] * 0.01
merged_df.loc[:, 'Hispanic_Incarcerated'] = merged_df['Hispanic'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['TotalPop'] * 0.01


# gets rid of the original demographic populations
clear_merged_df = merged_df.loc[:, ['State', 'Asian_Incarcerated', 'White_Incarcerated', 'Black_Incarcerated', 'Hispanic_Incarcerated', 'Native_Incarcerated']]

In [11]:
# groups by state 
final_df = clear_merged_df.groupby('State').sum()
final_df

,Asian_Incarcerated,White_Incarcerated,Black_Incarcerated,Hispanic_Incarcerated,Native_Incarcerated
State,,,,,
Alabama,706.172216,40389.872295,30768.961969,3267.570008,319.257351
Alaska,705.698784,5412.438181,352.953568,694.590189,2163.231683
Arizona,2446.213843,44883.818236,5168.337152,41399.882808,6349.955163
Arkansas,582.708007,26399.518616,12273.887986,3591.098042,217.593529
California,40443.017533,109211.092393,31440.325013,183171.094944,1464.159211
Colorado,2303.728257,48515.746142,5164.038077,25889.582024,563.297742
Connecticut,1661.285726,19550.320753,9999.699060,13117.229588,75.448626
Delaware,516.031693,10063.426611,6251.902516,2031.379873,55.606569
District of Columbia,548.087171,4865.747223,13484.554052,1862.070010,30.356484
